In [15]:
from scipy.spatial import distance #for euclidean distance
from imutils import face_utils #for finding facial landmarks
import imutils
import dlib
import cv2
import os

In [20]:
#finding euclidean distance between coordinates
#sqrt(a^2 + b^2)
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5]) 
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    #(vectical distance(A) + vectical distance(B))/(2*horizontal distance(C))
    ear = (A + B) / (2.0 * C)
    return ear

In [17]:
thresh = 0.25
frame_check = 60 #human visual system can process 10 to 12 images per second
#almost 6 seconds eyes are closed
detect = dlib.get_frontal_face_detector()#contains faces of humans in jpg
#detects face of human
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks(1).dat") #downloading all facial landmarks

In [18]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]#storing starting and ending points of left eye; 37, 42
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]#storing starting and ending points of right eye; 43, 48
#for alarm
duration = 1  # seconds
freq = 440  # Hz

In [19]:
cap=cv2.VideoCapture(0) #opens camera
flag=0
while True:
    ret, frame=cap.read() #reading frames
    frame = imutils.resize(frame, width=450) #resizing frame for better processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #frame is converted to gray scale
    subjects = detect(gray, 0) #cropping the facial area only
    for subject in subjects:
        shape = predict(gray, subject) #predicting all facial landmarks on the cropped face
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd] #getting all the left eye points in an array
        rightEye = shape[rStart:rEnd] #getting all the right eye points in an array
        leftEAR = eye_aspect_ratio(leftEye) #calculating aspect ratio for left eye
        rightEAR = eye_aspect_ratio(rightEye) #calculating aspect ratio for right eye
        ear = (leftEAR + rightEAR) / 2.0 #finding average of aspect ratios
        #The convex hull of a set of points is defined as the smallest convex polygon, that 
        #encloses all of the points in the set. 
        #It connects all the outer points to club all points in a polygon.
        leftEyeHull = cv2.convexHull(leftEye) 
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1) #drawing green countour in left eye
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1) #drawing green countour in right eye
        if ear < thresh: #eye is closed
            flag += 1
            if flag >= frame_check: #eye is closed for longer period of time(I have taken 6 seconds)
                cv2.putText(frame, "****************ALERT!****************", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) #In Red colored font Alert is printed
                cv2.putText(frame, "****************ALERT!****************", (10,325),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq)) #system tune
                #played for all frames whose values satisfy the conditions
        else:
            flag = 0
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
        break
